In [1]:
import os
import numpy as np # type: ignore
import pandas as pd # type: ignore
import LeafSLA # custom functions from LeafSLA.py – must be in same directory as this script (pipeline.ipynb), otherwise you have to use os.chdir to help it find the script

os.chdir("/Users/kericlamb/Documents/Work MacBook/Research/protocols/phenotyping_drought/leaf_lobing/") # directory with the file ./LeafSLA.py in it


In [2]:
# variable definitions needed for both pipelines 
img_folder = "mim_SLA" # folder name that has images to process (within the current working directory, e.g., /Users/kericlamb/.../leaf_lobing/photos)
sq_size = 1 # size of the square (one side) in the image used as the standard -- I've been using SH=1 and LB=2 (?)

# for SAM pipeline
model_type = "vit_b"  # "vit_h" (best), "vit_l" (medium), "vit_b" (smallest)
checkpoint = "~/Documents/SAM/sam_vit_b_01ec64.pth"
bbox_multiplier = 4 # 2 for Lynn; 4 for Stacy... mess around with it. has to do with size of the red square more than anything
leaf_position = "above" # where leaf is relative to the red square
perspective_shift = False # for SH, False is best. for LB 2023 images, True is best

# for algorithmic pipeline
saturation = 50 # saturation level. 50 for Stacy H., 70 for Lynn B.


### Running the SAM Pipeline:

In [ ]:
# read in images and create lists of the image names sans extension (img_names) and image full paths to the working directory (img_list)
img_list, img_names = LeafSLA.img_import(img_folder=img_folder, folders=False)
df, area_df = LeafSLA.new_data()

# run a for-loop for all images in the specified img_folder
for i in range(len(img_names)):
    # print only every ten images
    if i % 1 == 0: 
        perc = int((i/len(img_names))*100)
        print(f'Processing image: {img_names[i]} - {perc} %   ', end='\r')
    
    try: 
        # reads in image and finds loose bounding boxes for the leaf and red box using color thresholding and saves an image copy to ./bbox path
        leaf_array, box_array = LeafSLA.setup_image(img_src=img_list[i], img_folder=img_folder, img_name=img_names[i], color_ranges=LeafSLA.color_thresher(), 
                                                    sq_size=sq_size, leaf_position=leaf_position, bbox_multiplier=bbox_multiplier)

        # applies SAM – does not return an object but saves copy to the ./threshold path
        LeafSLA.SAM_image(img_src=img_list[i], img_folder=img_folder, img_name=img_names[i], leaf_array=leaf_array, box_array=box_array, model_type="vit_b", 
                        checkpoint="./SAM/models/sam_vit_b_01ec64.pth", device="mps")

        # applies minor blurring and edge smoothing (degree of which is controlled by erode and sigma (these are hidden but can be added to the function below))
        binary_image = LeafSLA.image_repair(img_folder=img_folder, img_name=img_names[i])

        # corrects for perspective using the red square and saves a copy to ./perspective path
        if perspective_shift == True:
            binary_image = LeafSLA.perspective_correction(binary_image, img_folder, img_name=img_names[i])

        # measures all relevant phenotypes and saves a copy to ./processed path
        df, area_df = LeafSLA.contour_measurement(img_src=img_list[i], img_folder=img_folder, img_name=img_names[i], image=binary_image, sq_size=sq_size,
                                                df=df, area_df=area_df, method="SAM")
    except Exception as e: 
        print(f"{img_names[i]} had the error: {e}")
        print("") # prevents multiple statement prints per line
    
# save all relevant data for each image AFTER all images have been processed
df.to_csv('./data/{0}_metadata.csv'.format(img_folder), index=False)
area_df.to_csv('./data/{0}_measurements_sam.csv'.format(img_folder), index=False)

### Running the Naive Pipeline
###### This pipe is more prone to error on difficult images, but works decently on simpler images

In [ ]:
# read in images and create lists of the image names sans extension (img_names) and image full paths to the working directory (img_list)
img_list, img_names = LeafSLA.img_import(img_folder=img_folder, folders=False)
df, area_df = LeafSLA.new_data()

# run a for-loop for all images in the specified img_folder
for i in range(len(img_names)):
    # print only every ten images
    if i % 1 == 0: 
        perc = int((i/len(img_names))*100)
        print(f'Processing image: {img_names[i]} - {perc} %   ', end='\r')

    # this pipeline is more prone to error, so nesting in try/except loop that will print error if things go south
    try: 
        # read in image and apply naïve pipeline
        LeafSLA.algo_pipeline(img_src=img_list[i], img_folder=img_folder, img_name=img_names[i], saturation=saturation)

        # applies minor blurring and edge smoothing (degree of which is controlled by erode and sigma (these are hidden but can be added to the function below))
        binary_image = LeafSLA.image_repair(img_folder=img_folder, img_name=img_names[i])

        # corrects for perspective using the red square and saves a copy to ./perspective path
        corrected_image = LeafSLA.perspective_correction(binary_image, img_folder, img_name=img_names[i])

        # measures all relevant phenotypes and saves a copy to ./processed path
        df, area_df = LeafSLA.contour_measurement(img_src=img_list[i], img_folder=img_folder, img_name=img_names[1], image=corrected_image, sq_size=sq_size,
                                                  df=df, area_df=area_df)
    except Exception as e: 
        print(f"{img_names[i]} had the error: {e}")
        print("") # prevents multiple statement prints per line

# save all relevant data for each image AFTER all images have been processed
df.to_csv('./data/{0}_metadata.csv'.format(img_folder), index=False)
area_df.to_csv('./data/{0}_measurements_sam.csv'.format(img_folder), index=False)
